---

# 🥉 LEVEL 1: SQL Warehouse Analyst

**Difficulty:** Intermediate | **Time:** 30 minutes | **Focus:** Databricks SQL

## 📚 What You'll Learn

- Creating and using SQL Warehouses
- Advanced SQL queries (CTEs, Window Functions, Aggregations)
- Multi-criteria decision analysis with SQL
- Weighted scoring models
- Business metric calculations

## 🎯 Your Mission

Use **Databricks SQL Warehouse** to analyze turbine data and create a priority ranking for maintenance decisions.

**Key SQL Concepts:**
- Common Table Expressions (CTEs) for complex logic
- Window functions for running calculations
- Normalization techniques (min-max scaling)
- Weighted scoring for multi-criteria decisions

---

## 📝 Task Instructions

### Step 0: Create Your SQL Warehouse (If needed)

**In Databricks UI:**
1. Click **SQL** in left sidebar
2. Go to **SQL Warehouses**
3. Click **Create SQL Warehouse** (if you don't have one)
4. Choose **Starter** (Free Edition)
5. Start the warehouse

💡 **Free Edition Note:** You get a Classic SQL Warehouse - perfect for this challenge!

---

### Step 1: Explore the Data

First, understand what data is available:

In [0]:
-- Check available tables
%sql
SHOW TABLES IN main.e2eai_iot_turbine;

In [0]:
-- Explore sensor data structure
%sql
SELECT * FROM main.e2eai_iot_turbine.sensor_hourly LIMIT 5;

In [0]:
-- Check turbine power metrics
%sql
SELECT * FROM main.e2eai_iot_turbine.turbine_power LIMIT 5;

### Step 2: Define Business Metrics

**Question:** What factors should influence maintenance priority?

Think about:
1. **Energy Output** - High-output turbines = more revenue at risk
2. **Failure Risk** - Likely to fail = urgent action needed
3. **Revenue Impact** - Lost revenue if turbine fails
4. **Operational Status** - Currently failing = critical priority

**Your Task:** Choose 3-4 metrics that matter for this business decision.

### 📝 Document Your Metric Selection

**Metric 1:** ________________________________  
**Why it matters:** ________________________________  
**How to calculate:** ________________________________  
**Weight (%):** ____%

**Metric 2:** ________________________________  
**Why it matters:** ________________________________  
**How to calculate:** ________________________________  
**Weight (%):** ____%

**Metric 3:** ________________________________  
**Why it matters:** ________________________________  
**How to calculate:** ________________________________  
**Weight (%):** ____%

**Total weights:** Should sum to 100%

### Step 3: Calculate Metrics with SQL

Build queries to calculate each metric. Use **CTEs** for clarity!

In [0]:
-- Metric 1: Energy Output Potential
%sql
CREATE OR REPLACE TEMP VIEW turbine_energy_output AS
SELECT
  turbine_id,
  AVG(avg_power) as avg_power_output,
  SUM(avg_power) as total_power_generated,
  COUNT(*) as hours_operational
FROM main.e2eai_iot_turbine.sensor_hourly
GROUP BY turbine_id;

-- View top performers
SELECT * FROM turbine_energy_output 
ORDER BY avg_power_output DESC 
LIMIT 10;

In [0]:
-- Metric 2: Failure Risk Score
%sql
CREATE OR REPLACE TEMP VIEW turbine_failure_risk AS
SELECT
  turbine_id,
  -- Abnormal readings indicate failure risk
  SUM(CASE WHEN avg_vibration > 1.5 THEN 1 ELSE 0 END) as high_vibration_hours,
  SUM(CASE WHEN avg_power < 500 THEN 1 ELSE 0 END) as low_power_hours,
  -- Risk score = % of abnormal readings
  (SUM(CASE WHEN avg_vibration > 1.5 OR avg_power < 500 THEN 1 ELSE 0 END) * 1.0 / COUNT(*)) as risk_score
FROM main.e2eai_iot_turbine.sensor_hourly
GROUP BY turbine_id;

-- View highest risk turbines
SELECT * FROM turbine_failure_risk 
ORDER BY risk_score DESC 
LIMIT 10;

In [0]:
-- Metric 3: Revenue at Risk
%sql
CREATE OR REPLACE TEMP VIEW turbine_revenue_risk AS
SELECT
  turbine_id,
  avg_power_output,
  -- Daily revenue: power * 24 hours * $0.05/kWh
  (avg_power_output * 24 * 0.05) as daily_revenue,
  -- Quarterly revenue at risk (90 days)
  (avg_power_output * 24 * 0.05 * 90) as quarterly_revenue_at_risk
FROM turbine_energy_output;

-- View highest revenue risk
SELECT * FROM turbine_revenue_risk 
ORDER BY quarterly_revenue_at_risk DESC 
LIMIT 10;

### Step 4: Build Weighted Scoring Model

**Key SQL Technique:** Min-Max Normalization

```sql
normalized_value = (value - MIN(value)) / (MAX(value) - MIN(value)) * 100
```

This scales all metrics to 0-100 range so they can be combined fairly.

In [0]:
-- Combined Priority Score with Normalization
%sql
CREATE OR REPLACE TEMP VIEW turbine_priority_scores AS
WITH normalized_metrics AS (
  SELECT
    e.turbine_id,
    e.avg_power_output,
    r.risk_score,
    rv.quarterly_revenue_at_risk,
    
    -- Normalize each metric to 0-100 scale
    ((e.avg_power_output - MIN(e.avg_power_output) OVER ()) / 
     NULLIF(MAX(e.avg_power_output) OVER () - MIN(e.avg_power_output) OVER (), 0) * 100) as normalized_power,
    
    (r.risk_score * 100) as normalized_risk,
    
    ((rv.quarterly_revenue_at_risk - MIN(rv.quarterly_revenue_at_risk) OVER ()) / 
     NULLIF(MAX(rv.quarterly_revenue_at_risk) OVER () - MIN(rv.quarterly_revenue_at_risk) OVER (), 0) * 100) as normalized_revenue
     
  FROM turbine_energy_output e
  JOIN turbine_failure_risk r ON e.turbine_id = r.turbine_id
  JOIN turbine_revenue_risk rv ON e.turbine_id = rv.turbine_id
)
SELECT
  turbine_id,
  avg_power_output,
  risk_score,
  quarterly_revenue_at_risk,
  
  -- Weighted Priority Score
  -- Adjust weights based on business priorities!
  (COALESCE(normalized_power, 0) * 0.25) +        -- 25% weight on energy output
  (COALESCE(normalized_risk, 0) * 0.50) +         -- 50% weight on failure risk (most important!)
  (COALESCE(normalized_revenue, 0) * 0.25)        -- 25% weight on revenue
  AS priority_score
  
FROM normalized_metrics;

-- View top priorities
SELECT 
  turbine_id,
  ROUND(priority_score, 2) as priority,
  ROUND(avg_power_output, 0) as avg_power_kw,
  ROUND(risk_score * 100, 1) as failure_risk_pct,
  ROUND(quarterly_revenue_at_risk, 0) as revenue_at_risk_usd
FROM turbine_priority_scores 
ORDER BY priority_score DESC 
LIMIT 10;

### Step 5: Generate TOP 10 Recommendation

Create final list with business justification:

In [0]:
-- Final TOP 10 Recommendation
%sql
SELECT
  turbine_id,
  ROUND(priority_score, 2) as priority_score,
  ROUND(avg_power_output, 0) as avg_power_kw,
  ROUND(risk_score * 100, 1) as failure_risk_pct,
  ROUND(quarterly_revenue_at_risk, 0) as revenue_at_risk_usd,
  
  -- Business justification
  CASE
    WHEN priority_score > 75 THEN '🔴 CRITICAL - Immediate action required'
    WHEN priority_score > 50 THEN '🟡 HIGH - Schedule within 2 weeks'
    WHEN priority_score > 25 THEN '🟠 MEDIUM - Schedule within month'
    ELSE '🟢 LOW - Monitor only'
  END as urgency_level,
  
  -- Expected action
  CASE
    WHEN priority_score > 75 THEN 'Emergency maintenance'
    WHEN priority_score > 50 THEN 'Preventive maintenance'
    ELSE 'Routine inspection'
  END as recommended_action
  
FROM turbine_priority_scores
ORDER BY priority_score DESC
LIMIT 10;

## ✅ Validation Steps

**Review your SQL work:**

1. **Data Quality Check:**
   - Do all turbines have scores? Check for NULLs
   - Are rankings reasonable?
   - Any extreme outliers?

2. **Business Logic Check:**
   - High-power turbines prioritized? ✓
   - High-risk turbines included? ✓
   - Revenue impact considered? ✓

3. **SQL Best Practices:**
   - Used CTEs for readability? ✓
   - Normalized metrics before combining? ✓
   - Handled NULL values (COALESCE)? ✓
   - Added business context (urgency levels)? ✓

## 🎓 SQL Techniques You Mastered

✅ **CTEs (Common Table Expressions)** - Breaking complex logic into readable steps  
✅ **Window Functions** - MIN/MAX OVER() for normalization  
✅ **Aggregations** - AVG, SUM, COUNT for business metrics  
✅ **CASE Statements** - Business logic and categorization  
✅ **NULLIF & COALESCE** - Handling edge cases  
✅ **JOINs** - Combining multiple data sources  
✅ **Temp Views** - Organizing multi-step analysis  

## 📊 Success Criteria

✅ Created SQL Warehouse  
✅ Explored data with SELECT queries  
✅ Defined 3+ business metrics  
✅ Calculated each metric with SQL  
✅ Normalized metrics to common scale  
✅ Built weighted scoring model  
✅ Generated TOP 10 turbine recommendation  
✅ Added business context (urgency, actions)  

**Level 1 Complete!** 🎉 You've mastered SQL-based business analytics!

---

**Next:** Level 2 - Use AI/BI Genie to explore data with natural language!

---

# 🥈 LEVEL 2: AI Data Explorer with Genie Spaces

**Difficulty:** Advanced | **Time:** 30 minutes | **Focus:** AI/BI Genie

## 📚 What You'll Learn

- Creating and configuring Genie Spaces
- Natural language query techniques
- Iterating on AI-generated queries
- Data discovery without SQL knowledge
- Teaching Genie about your business context

## 🎯 Your Mission

Use **AI/BI Genie Spaces** to explore turbine data through natural language questions. Validate your SQL analysis from Level 1 and discover new insights!

**What is Genie?**
- AI-powered data assistant
- Understands natural language questions
- Generates SQL queries automatically
- Learns from your Unity Catalog metadata
- Can be refined with feedback

---

## 📝 Task Instructions

### Step 1: Create Your Genie Space

**In Databricks UI:**

1. Click **Genie** in left sidebar (under SQL)
2. Click **Create Genie Space** or **Connect your data**
3. **Connect Data Sources:**
   - Select catalog: `main`
   - Select schema: `e2eai_iot_turbine`
   - Check tables: `sensor_hourly`, `turbine_power`, `turbine_status`
4. **Name your space:** "Wind Turbine Maintenance Analysis"
5. **Description:** "Analyzing wind turbine sensor data for maintenance prioritization"
6. Click **Create**

💡 **Tip:** Good descriptions help Genie understand context!

---

### Step 2: Basic Data Exploration

Ask Genie simple questions to understand the data:

**Try these questions:**

```
1. "How many turbines do we have in total?"

2. "Show me the average power output for each turbine"

3. "Which turbines have the highest vibration readings?"

4. "What's the distribution of power output across all turbines?"

5. "Show me turbines with power output above 2000 kW"
```

**For each question:**
- ✍️ Copy the question you asked
- 📊 Note the result
- 🔍 Review the SQL Genie created (click "Show SQL")
- ✅ Verify the answer makes sense

### 📝 Document Your Exploration

**Question 1:** ________________________________  
**Result:** ________________________________  
**SQL Generated:** (paste first few lines)  
**Insight:** ________________________________

**Question 2:** ________________________________  
**Result:** ________________________________  
**Insight:** ________________________________

*(Continue for all 5 questions)*

### Step 3: Business-Focused Questions

Now ask more complex business questions:

**Advanced Questions:**

```
6. "Which 10 turbines have the highest risk of failure based on abnormal sensor readings?"

7. "Calculate the potential revenue loss if each turbine goes down for 30 days"

8. "Show me turbines with both high power output and high vibration"

9. "Compare turbines with average power above 1500 kW"

10. "What percentage of turbines have vibration readings above 1.5?"
```

**Challenge:** Can Genie answer these without modification? Or do you need to:
- Rephrase the question?
- Break it into smaller questions?
- Edit the SQL Genie generated?

### Step 4: Validate Against SQL Analysis

**Cross-Check with Level 1 Results:**

Ask Genie: *"Show me the top 10 turbines by priority for maintenance, considering power output, failure risk, and revenue impact"*

**Compare:**
- Does Genie's list match your SQL analysis?
- Are the same turbines prioritized?
- If different, why? (different weights, different metrics?)

**Document differences:**
```
Genie's Top 5: ____________
Your SQL Top 5: ____________
Differences: ____________
Reason: ____________
```

### Step 5: Teach Genie About Business Context

**Improve Genie's Understanding:**

1. **Add Sample Questions** to your Genie Space:
   - Click "Settings" in Genie Space
   - Add "Sample Questions" like:
     - "Which turbines need urgent maintenance?"
     - "What's the ROI of maintaining each turbine?"
     - "Show me turbines at risk in the next quarter"

2. **Improve Table Descriptions** in Unity Catalog:
   - Go to **Catalog Explorer** → `main.e2eai_iot_turbine`
   - For each table, click **Edit** and add descriptions:
     - `sensor_hourly`: "Hourly aggregated sensor readings including vibration, temperature, power output"
   
3. **Add Column Comments:**
   - Click on `sensor_hourly` table
   - For key columns, add descriptions:
     - `avg_vibration`: "Average vibration in m/s² - values >1.5 indicate potential failure"
     - `avg_power`: "Average power output in kW"

4. **Re-ask questions** - Does Genie give better answers now?

## 🎓 Genie Techniques You Mastered

✅ **Natural Language Queries** - Asking data questions in plain English  
✅ **Iterative Refinement** - Improving queries through follow-ups  
✅ **SQL Review** - Understanding generated queries  
✅ **Context Enhancement** - Adding metadata to improve results  
✅ **Business Communication** - Framing questions for business insights  
✅ **Data Validation** - Cross-checking AI results against manual analysis  

## ✅ Success Criteria

✅ Created Genie Space with turbine data  
✅ Asked 10+ natural language questions  
✅ Reviewed SQL generated by Genie  
✅ Validated results against Level 1 analysis  
✅ Improved Genie with sample questions & descriptions  
✅ Documented what Genie does well & where it struggles  

## 💡 Reflection Questions

1. **What could Genie answer easily?**  
   (Simple aggregations, filtering, basic analysis)

2. **What did Genie struggle with?**  
   (Complex business logic, multi-step calculations, domain-specific context)

3. **When would you use Genie vs SQL?**  
   - Genie: Quick exploration, ad-hoc questions, non-technical users
   - SQL: Complex logic, reproducible analysis, performance-critical queries

**Level 2 Complete!** 🎉 You've mastered AI-powered data exploration!

---

**Next:** Level 3 - Build an executive dashboard!

---

# 🥇 LEVEL 3: Dashboard Designer

**Difficulty:** Expert | **Time:** 45 minutes | **Focus:** Databricks SQL Dashboards

## 📚 What You'll Learn

- Dashboard design principles for executives
- Creating visualizations (charts, tables)
- Dashboard layout and user experience
- Filters and interactivity
- Data storytelling
- Embedding Genie in dashboards

## 🎯 Your Mission

Create an **executive dashboard** that tells the story of your maintenance recommendation. The CEO will present this to the board tomorrow!

**Dashboard Requirements:**
- **Clear story:** Problem → Analysis → Recommendation
- **Executive-friendly:** No technical jargon
- **Actionable:** Clear next steps
- **Interactive:** Filters for exploration
- **Genie-enabled:** Board can ask questions

---

## 📝 Task Instructions

### Step 1: Plan Your Dashboard

**Dashboard Structure:**

```
┌──────────────────────────────────────────────────┐
│  WIND TURBINE MAINTENANCE PRIORITY DASHBOARD     │
│  Q1 Budget: $500K | Recommendation: 10 Turbines │
├──────────────────────┬───────────────────────────┤
│  📊 Chart 1:         │  📊 Chart 2:              │
│  Priority Heatmap    │  Risk vs Revenue Matrix   │
│  (Bar Chart)         │  (Scatter Plot)           │
├──────────────────────┼───────────────────────────┤
│  📊 Chart 3:         │  📊 Chart 4:              │
│  Regional Analysis   │  Top 10 Table             │
└──────────────────────┴───────────────────────────┘
```

**Key Visualizations:**
1. **Priority Heatmap** - Which turbines need attention?
2. **Risk Matrix** - Balance risk vs reward
3. **Regional Distribution** - Geographic clustering
4. **Top 10 Table** - Detailed recommendation

### Step 2: Create Dashboard in Databricks UI

**Instructions:**

1. Go to **SQL** → **Dashboards**
2. Click **Create Dashboard**
3. Name: "Q1 Turbine Maintenance Priority"

**For each visualization:**

#### Chart 1: Priority Bar Chart
- Query: Your `turbine_priority_scores` from Level 1
- Type: Bar Chart
- X: `turbine_id`
- Y: `priority_score`
- Color: By priority level (Red = Critical)

#### Chart 2: Risk-Revenue Scatter
- Query: Combine risk and revenue metrics
- Type: Scatter Plot
- X: `failure_risk_pct`
- Y: `revenue_at_risk_usd`
- Size: `priority_score`

#### Chart 3: Regional Analysis
- Query: Group turbines by region (use MOD function)
- Type: Grouped Bar Chart
- Show: Critical vs Normal turbines per region

#### Chart 4: Top 10 Table
- Query: Your TOP 10 recommendation from Level 1
- Type: Table
- Conditional formatting on urgency level

### Step 3: Add Interactivity

**Add Dashboard Filters:**

1. **Filter: Priority Level**
   - Multi-select: Critical, High, Medium, Low
   
2. **Filter: Minimum Priority Score**
   - Slider: 0-100

**Update queries to use filters** with parameters like:
```sql
WHERE priority_score >= {{ min_priority }}
```

### Step 4: Enable Genie in Dashboard

**Make Dashboard Conversational:**

1. In Dashboard Settings, click **Enable Genie**
2. Connect to your Genie Space from Level 2
3. A chat interface appears!

**Test Questions:**
- *"Why is this turbine ranked #1?"*
- *"Show me turbines with risk above 60%"*
- *"What's the average priority score?"*

💡 Board members can now explore data by asking questions!

### Step 5: Share Dashboard

**Finalize & Share:**

1. **Polish:**
   - Professional colors
   - Clear chart titles
   - Remove technical jargon
   
2. **Add Summary Card at top:**
   ```
   🚨 Q1 MAINTENANCE RECOMMENDATION
   
   Budget: $500K
   Action: Prioritize 10 turbines
   Expected Benefit: Prevent revenue loss
   
   ✅ Decision Required: Approve allocation
   ```

3. **Share:**
   - Click **Share**
   - Add CEO/Board members
   - Get shareable link

## 🎓 Dashboard Skills You Mastered

✅ **Dashboard Design** - Layout, color, hierarchy  
✅ **Data Visualization** - Charts that tell a story  
✅ **Executive Communication** - Business language  
✅ **Interactivity** - Filters for exploration  
✅ **Genie Integration** - Conversational analytics  

## ✅ Success Criteria

✅ Created Databricks SQL Dashboard  
✅ Added 4+ visualizations  
✅ Configured dashboard filters  
✅ Enabled Genie for Q&A  
✅ Professional design  
✅ Shared dashboard with permissions  
✅ Clear story: Problem → Analysis → Recommendation  

**Level 3 Complete!** 🏆 You've mastered executive dashboards!

---

## 🎯 Challenge Complete!

You've mastered the full BI workflow:

1. **🥉 SQL Warehouse** - Data analysis with advanced SQL
2. **🥈 Genie Spaces** - AI-powered data exploration  
3. **🥇 Dashboards** - Executive storytelling

**You're now prepared for BI/Data Analyst roles!** 💪

---

## 🚀 Next Steps

**Continue in This Repo:**
- **Module 04:** Machine Learning for predictive maintenance
- **Module 05:** Generative AI agents
- **Module 06:** Workflow orchestration

**Share Your Achievement:**
> Just completed an end-to-end BI challenge using Databricks! Mastered SQL Warehouses, AI/BI Genie, and executive dashboards. Solved a real business problem with data-driven prioritization. #BusinessIntelligence #Databricks #DataAnalytics

---

**Thank you!** 🎉 *Good BI drives ACTION, not just analysis.*

# Tutorials

Link: https://www.databricks.com/resources/demos/tutorials/aibi-analytics-in-capital-markets?itm_data=demo_center

### Databricks AI/BI: Analytics in Capital Markets with Dashboards and Genie

##### What You’ll Learn
Databricks AI/BI makes it easy for anyone in your organization to unlock the full potential of your marketing data. Once your data is ready and secured in Unity Catalog, through Data Intelligence, AI/BI automatically understands its structure, usage, and semantics, making it simple for BI developers to quickly build and share dashboards using Databricks Assistant and for business users to self-serve insights by conversing with their data in natural language.

In this demo, we'll load and show you how end users can easily analyze Analytics in Capital Markets by leveraging:

- AI/BI Dashboards with interactive visualizations to monitor and analyze key marketing metrics
- AI/BI Genie to go beyond dashboards and discover new insights through natural language

In [0]:
import dbdemos
dbdemos.install('aibi-portfolio-assistant', catalog='main', schema='dbdemos_aibi_fsi_portfolio_assistant')

### Databricks AI/BI: Customer Support Review with Dashboards and Genie

##### What You’ll Learn
Databricks AI/BI makes it easy for anyone in your organization to unlock the full potential of your customer support data. Once your data is ready and secured in Unity Catalog, through Data Intelligence, AI/BI automatically understands its structure, usage, and semantics. This makes it simple for BI developers to quickly build and share dashboards using Databricks Assistant and for business users to self-serve insights by conversing with their data in natural language.

In this demo, we'll load and show you how end users can easily analyze Customer Support Review by leveraging:

- AI/BI Dashboards with interactive visualizations to monitor and analyze key marketing metrics
- AI/BI Genie to go beyond dashboards and discover new insights through natural language

In [0]:
import dbdemos
dbdemos.install('aibi-customer-support', catalog='main', schema='dbdemos_aibi_customer_support')

### Databricks AI/BI: Genomic Patient Data Analysis with Dashboards and Genie

##### What You’ll Learn
Databricks AI/BI makes it easy for anyone in your organization to unlock the full potential of your genomic patient data. Once your data is ready and secured in Unity Catalog, through Data Intelligence, AI/BI automatically understands its structure, usage, and semantics. This makes it simple for BI developers to quickly build and share dashboards using Databricks Assistant and for business users to self-serve insights by conversing with their data in natural language.

In this demo, we'll load and show you how end users can easily analyze Genomic Patient Data Analysis by leveraging:

- AI/BI Dashboards with interactive visualizations to monitor and analyze key marketing metrics
- AI/BI Genie to go beyond dashboards and discover new insights through natural language

In [0]:
import dbdemos
dbdemos.install('aibi-patient-genomics', catalog='main', schema='dbdemos_aibi_hls_genomics')